In [12]:
from neuralnetlib.models import Transformer
from neuralnetlib.preprocessing import Tokenizer, pad_sequences
from neuralnetlib.losses import SequenceCrossEntropy
from neuralnetlib.optimizers import Adam
from neuralnetlib.callbacks import EarlyStopping

In [13]:
fr_sentences = [
    "bonjour.",
    "au revoir.",
    "merci beaucoup.",
    "s'il vous plaît.",
    "comment allez-vous ?",
    "je vais bien.",
    "je suis fatigué.",
    "je suis content.",
    "quel est votre nom ?",
    "mon nom est Jean.",
    "enchanté de vous rencontrer.",
    "bonne journée.",
    "bonne soirée.",
    "à demain.",
    "j'aime le café.",
    "je n'aime pas le thé.",
    "quelle heure est-il ?",
    "il est trois heures.",
    "où est la gare ?",
    "la gare est près d'ici.",
    "combien ça coûte ?",
    "c'est trop cher.",
    "parlez-vous anglais ?",
    "un peu.",
    "je ne comprends pas.",
    "pouvez-vous répéter ?",
    "je suis désolé.",
    "pas de problème.",
    "bon appétit.",
    "à votre santé.",
    "j'ai faim.",
    "j'ai soif.",
    "il fait beau aujourd'hui.",
    "il pleut.",
    "il fait froid.",
    "il fait chaud.",
    "je travaille ici.",
    "où habitez-vous ?",
    "j'habite à Paris.",
    "quel âge avez-vous ?",
    "j'ai vingt-cinq ans.",
    "avez-vous des frères et sœurs ?",
    "j'ai une sœur.",
    "j'ai un chat.",
    "j'aime voyager.",
    "je suis étudiant.",
    "je suis professeur.",
    "au secours !",
    "joyeux anniversaire !",
    "félicitations !"
]

en_sentences = [
    "hello.",
    "goodbye.",
    "thank you very much.",
    "please.",
    "how are you?",
    "i am fine.",
    "i am tired.",
    "i am happy.",
    "what is your name?",
    "my name is John.",
    "nice to meet you.",
    "have a nice day.",
    "have a good evening.",
    "see you tomorrow.",
    "i like coffee.",
    "i don't like tea.",
    "what time is it?",
    "it is three o'clock.",
    "where is the train station?",
    "the station is nearby.",
    "how much is it?",
    "it's too expensive.",
    "do you speak english?",
    "a little.",
    "i don't understand.",
    "can you repeat?",
    "i am sorry.",
    "no problem.",
    "enjoy your meal.",
    "cheers.",
    "i am hungry.",
    "i am thirsty.",
    "the weather is nice today.",
    "it's raining.",
    "it's cold.",
    "it's hot.",
    "i work here.",
    "where do you live?",
    "i live in Paris.",
    "how old are you?",
    "i am twenty-five years old.",
    "do you have brothers and sisters?",
    "i have a sister.",
    "i have a cat.",
    "i like to travel.",
    "i am a student.",
    "i am a teacher.",
    "help!",
    "happy birthday!",
    "congratulations!"
]

In [14]:
fr_tokenizer = Tokenizer(filters="")  # else the tokenizer would remove the special characters including ponctuation
en_tokenizer = Tokenizer(filters="")  # else the tokenizer would remove the special characters including ponctuation

fr_tokenizer.fit_on_texts(fr_sentences, preprocess_ponctuation=True)
en_tokenizer.fit_on_texts(en_sentences, preprocess_ponctuation=True)

In [15]:
x_train = fr_tokenizer.texts_to_sequences(fr_sentences, preprocess_ponctuation=True)
y_train = en_tokenizer.texts_to_sequences(en_sentences, preprocess_ponctuation=True)

max_len_x = max(len(seq) for seq in x_train)
max_len_y = max(len(seq) for seq in y_train)
max_seq_len = max(max_len_x, max_len_y)

vocab_size_fr = len(fr_tokenizer.word_index)
vocab_size_en = len(en_tokenizer.word_index)
max_vocab_size = max(vocab_size_fr, vocab_size_en)

In [16]:
# Verify all data
print(f"vocab_size_en: {vocab_size_en}, vocab_size_fr: {vocab_size_fr}")
print(f"max_len_x: {max_len_x}, max_len_y: {max_len_y}, max_vocab_size: {max_vocab_size}, max_seq_len: {max_seq_len}")
print("French sentences:")
print(fr_sentences)
print("English sentences:")
print(en_sentences)
print("French tokenizer:")
print(fr_tokenizer.word_index)
print("English tokenizer:")
print(en_tokenizer.word_index)

vocab_size_en: 95, vocab_size_fr: 107
max_len_x: 8, max_len_y: 8, max_vocab_size: 107, max_seq_len: 8
French sentences:
['bonjour.', 'au revoir.', 'merci beaucoup.', "s'il vous plaît.", 'comment allez-vous ?', 'je vais bien.', 'je suis fatigué.', 'je suis content.', 'quel est votre nom ?', 'mon nom est Jean.', 'enchanté de vous rencontrer.', 'bonne journée.', 'bonne soirée.', 'à demain.', "j'aime le café.", "je n'aime pas le thé.", 'quelle heure est-il ?', 'il est trois heures.', 'où est la gare ?', "la gare est près d'ici.", 'combien ça coûte ?', "c'est trop cher.", 'parlez-vous anglais ?', 'un peu.', 'je ne comprends pas.', 'pouvez-vous répéter ?', 'je suis désolé.', 'pas de problème.', 'bon appétit.', 'à votre santé.', "j'ai faim.", "j'ai soif.", "il fait beau aujourd'hui.", 'il pleut.', 'il fait froid.', 'il fait chaud.', 'je travaille ici.', 'où habitez-vous ?', "j'habite à Paris.", 'quel âge avez-vous ?', "j'ai vingt-cinq ans.", 'avez-vous des frères et sœurs ?', "j'ai une sœur."

In [22]:
model = Transformer(
    vocab_size=max_vocab_size,
    d_model=256,
    n_heads=8,
    n_encoder_layers=12,
    n_decoder_layers=8,
    d_ff=1024,
    dropout_rate=0.1,   
    max_sequence_length=max_seq_len,
    random_state=42,
    unk_idx=fr_tokenizer.word_index[fr_tokenizer.unk_token],
    sos_idx=fr_tokenizer.word_index[fr_tokenizer.sos_token],
    eos_idx=fr_tokenizer.word_index[fr_tokenizer.eos_token],
)


model.compile(
    loss_function=SequenceCrossEntropy(label_smoothing=0.1),
    optimizer=Adam(learning_rate=0.001),
    verbose=True
)

Transformer(
  vocab_size=108,
  d_model=256,
  n_heads=8,
  n_encoder_layers=12,
  n_decoder_layers=8,
  d_ff=1024,
  dropout_rate=0.1,
  max_sequence_length=8
)


In [23]:
x_train_padded, y_train_padded = model.prepare_data(x_train, y_train)

In [24]:
ea = EarlyStopping(monitor='loss', patience=10)

history = model.fit(
    x_train_padded, y_train_padded,
    epochs=200,
    batch_size=8,
    verbose=True,
    callbacks=[ea]
)

[==============================] 100% Epoch 1/200 - loss: 11.7474 -  - 3.96s
[==============================] 100% Epoch 2/200 - loss: 7.3801 -  - 3.84s
[==============================] 100% Epoch 3/200 - loss: 3.8941 -  - 3.81s
[==============================] 100% Epoch 4/200 - loss: 2.2796 -  - 3.84s
[==============================] 100% Epoch 5/200 - loss: 1.6991 -  - 3.82s
[==============================] 100% Epoch 6/200 - loss: 1.6262 -  - 3.81s
[==============================] 100% Epoch 7/200 - loss: 1.5124 -  - 3.75s
[==============================] 100% Epoch 8/200 - loss: 1.4651 -  - 3.80s
[==============================] 100% Epoch 9/200 - loss: 1.4233 -  - 3.82s
[==============================] 100% Epoch 10/200 - loss: 1.3839 -  - 3.79s
[==============================] 100% Epoch 11/200 - loss: 1.3492 -  - 3.84s
[==============================] 100% Epoch 12/200 - loss: 1.3191 -  - 3.77s
[==============================] 100% Epoch 13/200 - loss: 1.2940 -  - 3.80s
[======

c:\users\marcp\documents\programming\python\neuralnetlib\neuralnetlib\optimizers.py:160: RuntimeWarning: overflow encountered in square
  v = self.beta_2 * v + (1 - self.beta_2) * np.square(grad)


[==============================] 100% Epoch 32/200 - loss: 1.1444 -  - 3.85s
[==============================] 100% Epoch 33/200 - loss: 1.1412 -  - 3.83s
[==============================] 100% Epoch 34/200 - loss: 1.1389 -  - 3.90s
[==============================] 100% Epoch 35/200 - loss: 1.1359 -  - 3.80s
[==============================] 100% Epoch 36/200 - loss: 1.1336 -  - 3.88s
[==============================] 100% Epoch 37/200 - loss: 1.1307 -  - 3.87s
[==============================] 100% Epoch 38/200 - loss: 1.1279 -  - 3.87s
[==============================] 100% Epoch 39/200 - loss: 1.1251 -  - 3.88s
[==============================] 100% Epoch 40/200 - loss: 1.1232 -  - 3.81s
[==============================] 100% Epoch 41/200 - loss: 1.1215 -  - 3.84s
[==============================] 100% Epoch 42/200 - loss: 1.1198 -  - 3.82s
[==============================] 100% Epoch 43/200 - loss: 1.1179 -  - 3.89s
[==============================] 100% Epoch 44/200 - loss: 1.1148 -  - 3.85s

KeyboardInterrupt: 

In [25]:
def translate(sentence: str, model, fr_tokenizer, en_tokenizer, 
             temperature=0.8, beam_size=10, min_length=2) -> str:
    print(f"\nProcessing: {sentence}")
    
    # Tokenization
    tokens = fr_tokenizer.texts_to_sequences([sentence], preprocess_ponctuation=True)[0]
    print(f"Tokens: {tokens}")
    print(f"Original words: {[fr_tokenizer.index_word.get(t, '<UNK>') for t in tokens]}")
    
    # Add special tokens
    tokens = fr_tokenizer.encode_special_tokens([tokens])[0]
    print(f"With special tokens: {tokens}")
    
    # Padding
    padded = pad_sequences([tokens], max_length=model.max_sequence_length, padding='post', 
                          pad_value=fr_tokenizer.word_index[fr_tokenizer.pad_token])
    print(f"Padded sequence: {padded}")
    
    pred = model.predict(padded, 
                        max_length=model.max_sequence_length,
                        beam_size=beam_size,      
                        alpha=0.6,         
                        min_length=min_length,
                        temperature=temperature)[0]
    
    print(f"Raw prediction: {pred}")
    
    words = []
    for idx in pred[1:]:  # Skip SOS
        if idx == en_tokenizer.word_index[en_tokenizer.eos_token]:
            print(f"Found EOS token: {idx}")
            break
            
        if idx == en_tokenizer.word_index[en_tokenizer.pad_token]:
            print(f"Skipping PAD token: {idx}")
            continue
            
        word = en_tokenizer.index_word.get(idx, en_tokenizer.unk_token)
        print(f"Token {idx} -> word: {word}")
        words.append(word)
    
    if not words:
        print("Warning: Empty translation, using default handling...")
        for idx in pred[1:]:  # Skip SOS
            if idx not in [en_tokenizer.word_index[token] for token in 
                         [en_tokenizer.pad_token, en_tokenizer.eos_token]]:
                word = en_tokenizer.index_word.get(idx, en_tokenizer.unk_token)
                words.append(word)
    
    translation = " ".join(words) if words else "[Translation failed]"
    print(f"Final translation: {translation}")
    return translation

test_sentences = [
    "je vais bien",
    "comment allez-vous ?",
    "bonjour"
]

print("Vocabulary sizes:")
print(f"French vocab size: {len(fr_tokenizer.word_index)}")
print(f"English vocab size: {len(en_tokenizer.word_index)}")

for sent in test_sentences:
    print("\n" + "="*50)
    print(f"Testing: {sent}")
    translation = translate(sent, model, fr_tokenizer, en_tokenizer, 
                      temperature=0.8,
                      beam_size=10,
                      min_length=2) 
    print(f"Translation: {translation}")
    

Vocabulary sizes:
French vocab size: 107
English vocab size: 95

Testing: je vais bien

Processing: je vais bien
Tokens: [3, 37, 38]
Original words: ['je', 'vais', 'bien']
With special tokens: [106, 3, 37, 38, 107]
Padded sequence: [[106   3  37  38 107   0   0   0]]
Raw prediction: [106   1   1]
Token 1 -> word: .
Token 1 -> word: .
Final translation: . .
Translation: . .

Testing: comment allez-vous ?

Processing: comment allez-vous ?
Tokens: [35, 36, 5, 4, 2]
Original words: ['comment', 'allez', '-', 'vous', '?']
With special tokens: [106, 35, 36, 5, 4, 2, 107]
Padded sequence: [[106  35  36   5   4   2 107   0]]
Raw prediction: [106   1   1]
Token 1 -> word: .
Token 1 -> word: .
Final translation: . .
Translation: . .

Testing: bonjour

Processing: bonjour
Tokens: [29]
Original words: ['bonjour']
With special tokens: [106, 29, 107]
Padded sequence: [[106  29 107   0   0   0   0   0]]
Raw prediction: [106   1   1]
Token 1 -> word: .
Token 1 -> word: .
Final translation: . .
Translat